# Data Understanding – MoMo Top-up Case Study

In [ ]:
import pandas as pd
import re
from pathlib import Path

In [1]:
import sys
sys.executable

'c:\\Users\\ASUS\\Documents\\Project\\Momo case study\\.venv\\Scripts\\python.exe'

In [2]:
path = 'C:/Users/ASUS/Documents/Project/Momo case study/Data/raw/momo_top_up.xlsx'

In [3]:
df_transactions = pd.read_excel(path, sheet_name=0)
df_transactions.head(5)

,user_id,order_id,Date,Amount,Merchant_id,Purchase_status
0,21269588,4169517626,2020-01-01,"10,000",13,NaN
1,28097592,4170276686,2020-01-01,"20,000",13,NaN
2,47435144,4166729310,2020-01-01,"10,000",12,NaN
3,29080935,4174460303,2020-01-01,"10,000",13,NaN
4,14591075,4168216749,2020-01-01,"10,000",12,NaN


In [4]:
def normalize_date_string(series: pd.Series) -> pd.Series:
    def _normalize(x):
        if pd.isna(x):
            return pd.NA

        x = str(x).strip()

        m = re.match(r"^(\d{4})[-/](\d{1,2})[-/](\d{1,2})$", x)
        if m:
            y, mth, d = m.groups()
            return f"{y}-{int(mth):02d}-{int(d):02d}"
        m = re.match(r"^(\d{1,2})[-/](\d{1,2})[-/](\d{4})$", x)
        if m:
            d, mth, y = m.groups()
            return f"{y}-{int(mth):02d}-{int(d):02d}"

        return pd.NA

    return series.apply(_normalize)

In [5]:
df_transactions['Date'] = normalize_date_string(df_transactions['Date'])

In [6]:
df_transactions['Date'].value_counts()

Date
2020-06-30    79
2020-07-30    75
2020-11-30    75
2020-05-30    72
2020-06-10    70
              ..
2020-03-11    23
2020-03-06    22
2020-01-11    22
2020-01-12    21
2020-01-07    13
Name: count, Length: 366, dtype: int64

In [7]:
df_transactions['Date'] = pd.to_datetime(df_transactions['Date'],format='mixed',errors= 'coerce',dayfirst= True)
print(df_transactions['Date'].isnull().sum())

0


In [8]:
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13495 entries, 0 to 13494
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   user_id          13495 non-null  int64         
 1   order_id         13495 non-null  int64         
 2   Date             13495 non-null  datetime64[ns]
 3   Amount           13495 non-null  object        
 4   Merchant_id      13495 non-null  int64         
 5   Purchase_status  2235 non-null   object        
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 632.7+ KB


In [9]:
df_transactions['Amount'] = df_transactions['Amount'].astype(str).str.replace(',', '').astype(float)

In [10]:
df_transactions['Purchase_status'] = (df_transactions['Purchase_status'] == 'Mua hộ').astype(int)

## Quan sát df_user info ta thấy 


In [11]:
df_user_info    = pd.read_excel(path, sheet_name=2)
df_user_info.head(5)

,User_id,First_tran_date,Location,Age,Gender
0,41654498,2018-05-02,HCMC,33_to_37,MALE
1,51276281,2019-12-08,Other Cities,unknown,MALE
2,49152375,2019-12-20,Other Cities,23_to_27,MALE
3,5971050,2019-08-01,HCMC,28_to_32,MALE
4,48134464,2019-08-23,Other Cities,18_to_22,FEMALE


In [12]:
df_user_info['First_tran_date'] = normalize_date_string(df_user_info['First_tran_date'])

In [13]:
df_user_info['First_tran_date'] = pd.to_datetime(df_user_info['First_tran_date'].astype(str).str.strip(), errors='coerce')


print(df_user_info['First_tran_date'].isnull().sum())


74


In [14]:
print(df_user_info['Location'].value_counts(dropna=False).sort_index())


Location
HCMC                4112
HN                  1437
Ho Chi Minh City      63
Other               1033
Other Cities        6022
Unknown              761
Name: count, dtype: int64


In [15]:
mapping_location = {
    'Ho Chi Minh City': 'HCMC',   
    'Other': 'Other Cities',      
    'Unknown': 'Other Cities'     
}

df_user_info['Location'] = df_user_info['Location'].replace(mapping_location)

print(df_user_info['Location'].value_counts())

Location
Other Cities    7816
HCMC            4175
HN              1437
Name: count, dtype: int64


In [16]:
print(df_user_info['Gender'].value_counts(dropna=False).sort_index())

Gender
FEMALE    3407
M           92
MALE      6282
Nam       1370
Nữ        1248
f           55
female     903
male        71
Name: count, dtype: int64


In [17]:
df_user_info['Gender'] = df_user_info['Gender'].str.upper()

mapping_gender = {
    'NỮ': 'FEMALE',
    'F': 'FEMALE',
    'NAM': 'MALE',
    'M': 'MALE'
}
df_user_info['Gender'] = df_user_info['Gender'].replace(mapping_gender)
print(df_user_info['Gender'].value_counts())

Gender
MALE      7815
FEMALE    5613
Name: count, dtype: int64


In [22]:
df_commission = pd.read_excel(path, sheet_name=1)

In [26]:
out_dir = Path("C:/Users/ASUS/Documents/Project/Momo case study/Data/processed")
out_dir.mkdir(parents=True, exist_ok=True)
df_transactions.to_csv(out_dir / "df_transactions.csv", index=False, encoding="utf-8-sig")
df_user_info.to_csv(out_dir / "df_users.csv",        index=False, encoding="utf-8-sig")
df_commission.to_csv(out_dir / "df_products.csv",     index=False, encoding="utf-8-sig")